In [1]:
import sys
import ROOT
import numpy as np
import pandas as pd
import root_pandas
import seaborn as sb
import matplotlib.pyplot as plt
import uproot
import time

from itertools import product

#from root_numpy import root2array

from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc

Welcome to JupyROOT 6.16/00


/Users/jonamotta/anaconda3/envs/python_root/lib/python2.7/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.16.3 is currently installed but you installed root_numpy against numpy 1.9.3. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)
Using TensorFlow backend.


In [2]:
########################################
### READ TREES AND CREATE DATAFRAMES ###
########################################


# fix random seed for reproducibility
np.random.seed(1986)

#create DataFrames with the values coming from the trees
file_tau = uproot.open('../bc_jpsi_tau_nu_gen_v2.root')
tree_tau = file_tau['tree;1']
tau  = tree_tau.pandas.df(tree_tau.keys())

file_mu = uproot.open('../bc_jpsi_mu_nu_gen_v2.root')
tree_mu = file_mu['tree;2']
mu  = tree_mu.pandas.df(tree_mu.keys())

In [3]:
#####################################
### DO THE CUTS TO THE DATAFRAMES ###
#####################################


tau_cut = pd.DataFrame()

k = 0
for j in tau.index:
    if tau.at[j,'mu1_eta'] < -1 or tau.at[j,'mu1_eta'] > 0.5:
        tau_cut.at[k,'bc_pt'] = tau.at[j,'bc_pt']
        tau_cut.at[k,'bc_eta'] = tau.at[j,'bc_eta']
        tau_cut.at[k,'bc_phi'] = tau.at[j,'bc_phi']
        tau_cut.at[k,'mu1_pt'] = tau.at[j,'mu1_pt']
        tau_cut.at[k,'mu1_eta'] = tau.at[j,'mu1_eta']
        tau_cut.at[k,'mu1_phi'] = tau.at[j,'mu1_phi']
        tau_cut.at[k,'mu1_charge'] = tau.at[j,'mu1_charge']
        tau_cut.at[k,'mu2_pt'] = tau.at[j,'mu2_pt']
        tau_cut.at[k,'mu2_eta'] = tau.at[j,'mu2_eta']
        tau_cut.at[k,'mu2_phi'] = tau.at[j,'mu2_phi']
        tau_cut.at[k,'mu2_charge'] = tau.at[j,'mu2_charge']
        tau_cut.at[k,'mu_pt'] = tau.at[j,'mu_pt']
        tau_cut.at[k,'mu_eta'] = tau.at[j,'mu_eta']
        tau_cut.at[k,'mu_phi'] = tau.at[j,'mu_phi']
        tau_cut.at[k,'mu_charge'] = tau.at[j,'mu_charge']
        tau_cut.at[k,'pv_x'] = tau.at[j,'pv_x']
        tau_cut.at[k,'pv_y'] = tau.at[j,'pv_y']
        tau_cut.at[k,'pv_z'] = tau.at[j,'pv_z']
        tau_cut.at[k,'sv_x'] = tau.at[j,'sv_x']
        tau_cut.at[k,'sv_y'] = tau.at[j,'sv_y']
        tau_cut.at[k,'sv_z'] = tau.at[j,'sv_z']
        k += 1

In [4]:
############################################
### ADD NECESSARY FEATURES TO DATAFRAMES ###
############################################


#add the column target to both dataframes
mu ['target'] = 0
tau_cut['target'] = 1
#add the columns of interesting features (taken from LHCb paper)
mu['m2_miss'] = 0 #missing mass square (p_B - p_mu1 - p_mu2 - p_mu)^2
tau_cut['m2_miss'] = 0
mu['muE_Brf'] = 0 #mu energy in the Bc rest frame
tau_cut['muE_Brf'] = 0
mu['q2'] = 0 #squared 4momentum transfer to lepton sys (p_B - p_mu1 - p_mu2)^2
tau_cut['q2'] = 0
mu['pT_miss'] = 0 #missing transverse momentum (p_B - p_mu1 - p_mu2 - p_mu).Pt
tau_cut['pT_miss'] = 0

bc_vect = ROOT.TLorentzVector()
jpsi_vect = ROOT.TLorentzVector()
mu_vect = ROOT.TLorentzVector()
mu1_vect = ROOT.TLorentzVector()
mu2_vect = ROOT.TLorentzVector()
reco_vect = ROOT.TLorentzVector()
PV = ROOT.TVector3()
SV = ROOT.TVector3()
mu_mass = 0.10565837 #GeV/c^2
jpsi_mass = 3.096900 #GeV/c^2
bc_mass = 6.2756 #GeV/c^2
c = 2.99e8 #m/s

# in the following, when creating the TLorentzVector of mu, mu1, mu2 we try to account for the reconstruction error
# made by the tracker and the muon system in the value of pT
# we include a gaussian smearing -> we draw a value of pT from a gaussian centered at the value of the MC pT of the
#                                   muon and with sigma 7% of the value of the MC pT (CMS performance)
start = time.time()
i = 0
while i < mu['run'].count(): 
    # set the reconstructed particles 4vectors considering the smearing in the measure of pT due to the 
    # experimental setup (considered to be 7% as a mean between the 1-1.5% at 10GeV and the 6-17% at 1TeV)
    mu_vect.SetPtEtaPhiM(max(np.random.normal(mu.at[i,'mu_pt'], mu.at[i,'mu_pt']*7/100),0),mu.at[i,'mu_eta'],mu.at[i,'mu_phi'],mu_mass)
    mu1_vect.SetPtEtaPhiM(max(np.random.normal(mu.at[i,'mu1_pt'], mu.at[i,'mu1_pt']*7/100),0),mu.at[i,'mu1_eta'],mu.at[i,'mu1_phi'],mu_mass)
    mu2_vect.SetPtEtaPhiM(max(np.random.normal(mu.at[i,'mu2_pt'], mu.at[i,'mu2_pt']*7/100),0),mu.at[i,'mu2_eta'],mu.at[i,'mu2_phi'],mu_mass)

    # set the PV and SV and calculate primary-secondary vertex distance
    # include smearing of PV reconstruction with a gaussian smearing taken from a CMS paper
    # include smearing of SV reconstruction with a gaussian smearing with sigma double of the PV one
    PV.SetXYZ(np.random.normal(mu.at[i,'pv_x'], 20*1e-6),np.random.normal(mu.at[i,'pv_y'], 20*1e-6),np.random.normal(mu.at[i,'pv_z'], 30*1e-6))
    SV.SetXYZ(np.random.normal(mu.at[i,'sv_x'], 40*1e-6),np.random.normal(mu.at[i,'sv_y'], 40*1e-6),np.random.normal(mu.at[i,'sv_z'], 60*1e-6))
    dist_PSV = np.sqrt((PV.X()-SV.X())**2+(PV.Y()-SV.Y())**2+(PV.Z()-SV.Z())**2)*1e-2
    
    '''
    # setting the above 4vectors and verteces without the smearing (un-comment it if needed)
    mu_vect.SetPtEtaPhiM(mu.at[i,'mu_pt'],mu.at[i,'mu_eta'],mu.at[i,'mu_phi'],mu_mass)
    mu1_vect.SetPtEtaPhiM(mu.at[i,'mu1_pt'],mu.at[i,'mu1_eta'],mu.at[i,'mu1_phi'],mu_mass)
    mu2_vect.SetPtEtaPhiM(mu.at[i,'mu2_pt'],mu.at[i,'mu2_eta'],mu.at[i,'mu2_phi'],mu_mass)
    PV.SetXYZ(mu.at[i,'pv_x'], mu.at[i,'pv_y'], mu.at[i,'pv_z'])
    SV.SetXYZ(mu.at[i,'sv_x'], mu.at[i,'sv_y'], mu.at[i,'sv_z'])
    dist_PSV = np.sqrt((PV.X()-SV.X())**2+(PV.Y()-SV.Y())**2+(PV.Z()-SV.Z())**2)*1e-2
    '''
    
    reco_vect = mu_vect + mu1_vect + mu2_vect
    
    # reconstruct the pT of the Bc and set the 4vector
    bc_pZreco = (bc_mass / reco_vect.Mag()) * reco_vect.Pz()
    bc_pXreco = bc_pZreco*(SV.X()-PV.X())/(SV.Z() - PV.Z())
    bc_pYreco = bc_pZreco*(SV.Y()-PV.Y())/(PV.Z()-SV.Z())
    bc_Preco = bc_pZreco * dist_PSV / ((SV.Z() - PV.Z())*1e-2)
    bc_vect.SetPtEtaPhiM(np.sqrt(bc_pXreco**2+bc_pYreco**2),mu.at[i,'bc_eta'],mu.at[i,'bc_phi'],bc_mass)
    
    #######
    # test using the reconstruction of pT instead of pZ (gives slightly different results) 
    #bc_pTreco = mu.at[i,'bc_mass'] * reco_vect.Pt() / reco_vect.Mag() 
    #bc_vect.SetPtEtaPhiM(mu.at[i,'bc_mass']*reco_vect.Pt()/reco_vect.Mag(),mu.at[i,'bc_eta'],mu.at[i,'bc_phi'],mu.at[i,'bc_mass'])
    #######
    
    m2_vect = bc_vect - mu1_vect - mu2_vect - mu_vect
    q2_vect = bc_vect - mu1_vect - mu2_vect
    
    mu.at[i,'m2_miss'] = m2_vect.Mag2()
    mu.at[i,'q2'] = q2_vect.Mag2()
    mu.at[i,'muE_Brf'] = mu_vect.E() * np.cosh(mu_vect.Rapidity() - bc_vect.Rapidity())
    mu.at[i,'pT_miss'] = m2_vect.Pt()
    
    # decay length and time
    mu.at[i,'bc_DL'] = dist_PSV
    mu.at[i,'bc_CT'] = dist_PSV / (bc_vect.Gamma()*bc_vect.Beta())                     
    
    # we look for the best roconstruction of the jpsi 
    muons = [mu_vect,mu1_vect,mu2_vect]
    muons_charge = [mu.at[i,'mu_charge'],mu.at[i,'mu1_charge'],mu.at[i,'mu2_charge']]
    pair_vect = ROOT.TLorentzVector()
    if muons_charge[0] != muons_charge[1]:
        pair1_vect = muons[0] + muons[1]
        pair2_vect = muons[1] + muons[2]
    else:
        pair1_vect = muons[0] + muons[2]
        pair2_vect = muons[1] + muons[2]
     
    if (abs(pair1_vect.Mag() - jpsi_mass)) < abs((pair2_vect.Mag() - jpsi_mass)):
        mu.at[i,'muon_pair'] = 0 # indicating the best reco is mu+mu1/2
    else:
        mu.at[i,'muon_pair'] = 1 # indicating the best reco is mu1+mu2
    
    i += 1

                               
i = 0
while i < tau['run'].count():
    # set the reconstructed particles 4vectors considering the smearing in the measure of pT due to the 
    # experimental setup (considered to be 7% as a mean between the 1-1.5% at 10GeV and the 6-17% at 1TeV)
    mu_vect.SetPtEtaPhiM(max(np.random.normal(tau_cut.at[i,'mu_pt'], tau.at[i,'mu_pt']*7/100),0),tau_cut.at[i,'mu_eta'],tau_cut.at[i,'mu_phi'],mu_mass)
    mu1_vect.SetPtEtaPhiM(max(np.random.normal(tau_cut.at[i,'mu1_pt'], tau.at[i,'mu1_pt']*7/100),0),tau_cut.at[i,'mu1_eta'],tau_cut.at[i,'mu1_phi'],mu_mass)
    mu2_vect.SetPtEtaPhiM(max(np.random.normal(tau_cut.at[i,'mu2_pt'], tau.at[i,'mu2_pt']*7/100),0),tau_cut.at[i,'mu2_eta'],tau_cut.at[i,'mu2_phi'],mu_mass)

    # set the PV and SV and calculate primary-secondary vertex distance
    # include smearing of PV reconstruction with a gaussian smearing taken from a CMS paper
    # include smearing of SV reconstruction with a gaussian smearing with sigma double of the PV one
    PV.SetXYZ(np.random.normal(tau_cut.at[i,'pv_x'], 20*1e-6),np.random.normal(tau_cut.at[i,'pv_y'], 20*1e-6),np.random.normal(tau_cut.at[i,'pv_z'], 30*1e-6))
    SV.SetXYZ(np.random.normal(tau_cut.at[i,'sv_x'], 40*1e-6),np.random.normal(tau_cut.at[i,'sv_y'], 40*1e-6),np.random.normal(tau_cut.at[i,'sv_z'], 60*1e-6))
    dist_PSV = np.sqrt((PV.X()-SV.X())**2+(PV.Y()-SV.Y())**2+(PV.Z()-SV.Z())**2)*1e-2
    
    '''
    # setting the above 4vectors and verteces without the smearing (un-comment it if needed)
    mu_vect.SetPtEtaPhiM(tau.at[i,'mu_pt'],tau.at[i,'mu_eta'],tau.at[i,'mu_phi'],mu_mass)
    mu1_vect.SetPtEtaPhiM(tau.at[i,'mu1_pt'],tau.at[i,'mu1_eta'],tau.at[i,'mu1_phi'],mu_mass)
    mu2_vect.SetPtEtaPhiM(tau.at[i,'mu2_pt'],tau.at[i,'mu2_eta'],tau.at[i,'mu2_phi'],mu_mass)
    PV.SetXYZ(tau.at[i,'pv_x'], tau.at[i,'pv_y'], tau.at[i,'pv_z'])
    SV.SetXYZ(tau.at[i,'sv_x'], tau.at[i,'sv_y'], tau.at[i,'sv_z'])
    dist_PSV = np.sqrt((PV.X()-SV.X())**2+(PV.Y()-SV.Y())**2+(PV.Z()-SV.Z())**2)*1e-2
    '''
    
    reco_vect = mu_vect + mu1_vect + mu2_vect
    
    # reconstruct the pT of the Bc and set the 4vector
    bc_pZreco = (bc_mass / reco_vect.Mag()) * reco_vect.Pz()
    bc_pXreco = bc_pZreco*(SV.X()-PV.X())/(SV.Z() - PV.Z())
    bc_pYreco = bc_pZreco*(SV.Y()-PV.Y())/(PV.Z()-SV.Z())
    bc_Preco = bc_pZreco * dist_PSV / ((SV.Z() - PV.Z())*1e-2)
    bc_vect.SetPtEtaPhiM(np.sqrt(bc_pXreco**2+bc_pYreco**2),tau_cut.at[i,'bc_eta'],tau_cut.at[i,'bc_phi'],bc_mass)
    
    ########
    # test using the reconstruction of pT instead of pZ (gives slightly different results) 
    #bc_pTreco = (tau.at[i,'bc_mass'] / reco_vect.Mag()) * reco_vect.Pt()
    #bc_vect.SetPtEtaPhiM(bc_pTreco,tau.at[i,'bc_eta'],tau.at[i,'bc_phi'],tau.at[i,'bc_mass'])
    ########
    
    m2_vect = bc_vect - mu1_vect - mu2_vect - mu_vect
    q2_vect = bc_vect - mu1_vect - mu2_vect
    
    tau_cut.at[i,'m2_miss'] = m2_vect.Mag2()
    tau_cut.at[i,'q2'] = q2_vect.Mag2()
    tau_cut.at[i,'muE_Brf'] = mu_vect.E() * np.cosh(mu_vect.Rapidity() - bc_vect.Rapidity())
    tau_cut.at[i,'pT_miss'] = m2_vect.Pt()
    
    # decay length and decay time(c*tau)
    tau_cut.at[i,'bc_DL'] = dist_PSV
    tau_cut.at[i,'bc_CT'] = dist_PSV / (bc_vect.Gamma()*bc_vect.Beta())
                          
    # we look for the best roconstruction of the jpsi 
    muons = [mu_vect,mu1_vect,mu2_vect]
    muons_charge = [mu.at[i,'mu_charge'],mu.at[i,'mu1_charge'],mu.at[i,'mu2_charge']]
    pair_vect = ROOT.TLorentzVector()
    if muons_charge[0] != muons_charge[1]:
        pair1_vect = muons[0] + muons[1]
        pair2_vect = muons[1] + muons[2]
    else:
        pair1_vect = muons[0] + muons[2]
        pair2_vect = muons[1] + muons[2]
     
    if (abs(pair1_vect.Mag() - jpsi_mass)) < (abs(pair2_vect.Mag() - jpsi_mass)):
        tau_cut.at[i,'muon_pair'] = 0 # indicating the best reco is mu+mu1/2
    else:
        tau_cut.at[i,'muon_pair'] = 1 # indicating the best reco is mu1+mu2
    
    i += 1
    
end = time.time()
print 'Running time to add the new vars to the df = %.1f'%(end - start)

KeyError: 136149

In [ ]:
#########################################
### PREPARE DFs FOR TRAINING AND TEST ###
#########################################


# some of these features are taken from the LHCb paper and have to be computed and added to the dataframes
features = [
    'm2_miss', 
    'muE_Brf', 
    'q2', 
    'pT_miss',
    'mu_pt'     ,
    'mu_eta'    ,
    'mu_phi'    ,
    #'mu_charge' ,
    'mu1_pt'    ,
    'mu1_eta'   ,
    'mu1_phi'   ,
    #'mu1_charge',
    'mu2_pt'    ,
    'mu2_eta'   ,
    'mu2_phi'   ,
    #'mu2_charge',
]

# concatenate the two samples
dataset = pd.concat([mu, tau_cut],sort=False)

# shuffle and split train/test
train, test = train_test_split(dataset, test_size=0.85, random_state=1986, shuffle=True)

# X and Y on the training sample
X = pd.DataFrame(train, columns=features)
Y = pd.DataFrame(train, columns=['target'])

In [ ]:
########################
### DEFINE THE MODEL ###
########################

# define the model
model = Sequential()
model.add(Dense(12, input_dim=len(features), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#######################
### TRAIN THE MODEL ###
#######################


# fit the model
model.fit(X, Y, epochs=10, batch_size=10, validation_split=0.5)

In [ ]:
######################################
### TEST THE TRAINING OF THE MODEL ###
######################################


# evaluate the model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# calculate predictions on the test sample
print 'predicting on', test.shape[0], 'events'
x = pd.DataFrame(test, columns=features)
y = model.predict(x)

# add the score to the test sample dataframe
test.insert(len(test.columns), 'nn_score', y)

In [ ]:
#################
### ROC CURVE ###
#################


# let sklearn do the heavy lifting and compute the ROC curves for you
fpr, tpr, wps = roc_curve(test.target, test.nn_score)

In [ ]:
##########################
### ROC & SCORE CURVES ###
##########################


# plot
plt.plot(fpr, tpr, label=r'$\mu$ vs. $\tau$ NN', color='b')
# plot also the diagonal, that corresponds to random picks, no discrimination power
xy = [i*j for i,j in product([10.**i for i in range(-8, 0)], [1,2,4,8])]+[1]
plt.plot(xy, xy, color='grey', linestyle='--', label='no-discrimination curve')
# cosmetics
plt.title('ROC curve')
plt.xlabel(r'$\epsilon(\tau)$')
plt.ylabel(r'$\epsilon(\mu)$')
# axis range
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
# grid
plt.grid(True)
# legend
plt.legend(loc='best')
# save figure and then clean it
plt.savefig('cutTest_roc.pdf')
plt.show()
#plt.clf()

# plot the discriminator shape for the muon and tau channel
test_tau = test[test.tau_pt>=0]
test_mu  = test[test.tau_pt <0]
sb.distplot(test_tau['nn_score'], kde=False, norm_hist=True, label=r'$\tau\to\mu$ channel', color='b')
sb.distplot(test_mu ['nn_score'], kde=False, norm_hist=True, label=r'$\mu$ channel' , color='r')
plt.title('Classifier distribution')
plt.xlabel('classifier score')
plt.ylabel('a.u.')
plt.xlim([0.0, 1.0])
#plt.ylim([0.0, 0.25])
plt.legend(loc='upper right')
plt.savefig('cutTest_score.pdf')
plt.show()
plt.clf()

In [ ]:
# compute the auc
auroc = auc(fpr, tpr)
print "AUC =", auroc 

# compute Gini index
print "Gini index =", (auroc-0.5)*2

In [ ]:
sb.distplot(mu['muE_Brf'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_cut['muE_Brf'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('E$_{\mu}$ in B$_c$ rest frame[GeV]')
plt.ylabel('n_ev')
plt.show()

sb.distplot(mu['q2'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_cut['q2'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('$Q^2$[GeV]')
plt.ylabel('n_ev')
plt.show()

sb.distplot(mu['pT_miss'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_cut['pT_miss'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('missing pT [GeV]')
plt.ylabel('n_ev')
plt.show()

sb.distplot(mu['m2_miss'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_cut['m2_miss'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('missing $m^2$ [GeV]')
plt.ylabel('n_ev')
#plt.ylim([0,3e-8])
plt.show()

In [ ]:
mu_wrong = pd.DataFrame()
tau_wrong = pd.DataFrame()

# columns=['muon_pair','mu1_pt','mu1_eta','mu1_phi','mu1_charge','mu2_pt','mu2_eta','mu2_phi','mu2_charge','mu_pt','mu_eta','mu_phi','mu_charge','m2_miss','muE_Brf','q2','pT_miss']

#print test_mu.columns.values.tolist()

i = 0
k = 0
mu_top = test_mu.head(1000)
tau_top = test_tau.head(1000)
#print mu_top
for j in test_mu.index:
    if test_mu.at[j,'nn_score'] > 0.4:
        mu_wrong.at[i,'mu1_pt'] = test_mu.at[j,'mu1_pt']
        mu_wrong.at[i,'mu1_eta'] = test_mu.at[j,'mu1_eta']
        mu_wrong.at[i,'mu1_phi'] = test_mu.at[j,'mu1_phi']
        mu_wrong.at[i,'mu1_charge'] = test_mu.at[j,'mu1_charge']
        mu_wrong.at[i,'mu2_pt'] = test_mu.at[j,'mu2_pt']
        mu_wrong.at[i,'mu2_eta'] = test_mu.at[j,'mu2_eta']
        mu_wrong.at[i,'mu2_phi'] = test_mu.at[j,'mu2_phi']
        mu_wrong.at[i,'mu2_charge'] = test_mu.at[j,'mu2_charge']
        mu_wrong.at[i,'mu_pt'] = test_mu.at[j,'mu_pt']
        mu_wrong.at[i,'mu_eta'] = test_mu.at[j,'mu_eta']
        mu_wrong.at[i,'mu_phi'] = test_mu.at[j,'mu_phi']
        mu_wrong.at[i,'mu_charge'] = test_mu.at[j,'mu_charge']
        mu_wrong.at[i,'m2_miss'] = test_mu.at[j,'m2_miss']
        mu_wrong.at[i,'pT_miss'] = test_mu.at[j,'pT_miss']
        mu_wrong.at[i,'q2'] = test_mu.at[j,'q2']
        mu_wrong.at[i,'muE_Brf'] = test_mu.at[j,'muE_Brf']
        mu_wrong.at[i,'muon_pair'] = test_mu.at[j,'muon_pair']
        i += 1
for j in test_tau.index:
    if test_tau.at[j,'nn_score'] < 0.4:
        tau_wrong.at[k,'mu1_pt'] = test_tau.at[j,'mu1_pt']
        tau_wrong.at[k,'mu1_eta'] = test_tau.at[j,'mu1_eta']
        tau_wrong.at[k,'mu1_phi'] = test_tau.at[j,'mu1_phi']
        tau_wrong.at[k,'mu1_charge'] = test_tau.at[j,'mu1_charge']
        tau_wrong.at[k,'mu2_pt'] = test_tau.at[j,'mu2_pt']
        tau_wrong.at[k,'mu2_eta'] = test_tau.at[j,'mu2_eta']
        tau_wrong.at[k,'mu2_phi'] = test_tau.at[j,'mu2_phi']
        tau_wrong.at[k,'mu2_charge'] = test_tau.at[j,'mu2_charge']
        tau_wrong.at[k,'mu_pt'] = test_tau.at[j,'mu_pt']
        tau_wrong.at[k,'mu_eta'] = test_tau.at[j,'mu_eta']
        tau_wrong.at[k,'mu_phi'] = test_tau.at[j,'mu_phi']
        tau_wrong.at[k,'mu_charge'] = test_tau.at[j,'mu_charge']
        tau_wrong.at[k,'m2_miss'] = test_tau.at[j,'m2_miss']
        tau_wrong.at[k,'pT_miss'] = test_tau.at[j,'pT_miss']
        tau_wrong.at[k,'q2'] = test_tau.at[j,'q2']
        tau_wrong.at[k,'muE_Brf'] = test_tau.at[j,'muE_Brf']
        tau_wrong.at[k,'muon_pair'] = test_tau.at[j,'muon_pair']
        k += 1

In [ ]:
sb.distplot(mu_wrong['muE_Brf'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_wrong['muE_Brf'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('E$_{\mu}$ in B$_c$ rest frame[GeV]')
plt.ylabel('n_ev')
plt.show()

sb.distplot(mu_wrong['q2'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_wrong['q2'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('$Q^2$[GeV]')
plt.ylabel('n_ev')
plt.show()

sb.distplot(mu_wrong['pT_miss'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_wrong['pT_miss'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('missing pT [GeV]')
plt.ylabel('n_ev')
plt.show()

sb.distplot(mu_wrong['m2_miss'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_wrong['m2_miss'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('missing $m^2$ [GeV]')
plt.ylabel('n_ev')
#plt.ylim([0,3e-8])
plt.show()

In [ ]:
sb.distplot(mu_wrong['mu_pt'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_wrong['mu_pt'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('$\mu$ pT [GeV]')
plt.ylabel('n_ev')
plt.show()

sb.distplot(mu_wrong['mu_eta'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_wrong['mu_eta'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('$\mu$ eta')
plt.ylabel('n_ev')
plt.show()

sb.distplot(mu_wrong['mu_phi'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_wrong['mu_phi'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('$\mu$ phi')
plt.ylabel('n_ev')
plt.show()

In [ ]:
sb.distplot(mu_wrong['mu1_pt'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_wrong['mu1_pt'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('$\mu1$ pT [GeV]')
plt.ylabel('n_ev')
plt.show()

sb.distplot(mu_wrong['mu1_eta'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_wrong['mu1_eta'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('$\mu1$ eta')
plt.ylabel('n_ev')
plt.show()

sb.distplot(mu_wrong['mu1_phi'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_wrong['mu1_phi'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('$\mu1$ phi')
plt.ylabel('n_ev')
plt.show()

In [ ]:
sb.distplot(mu_wrong['mu2_pt'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_wrong['mu2_pt'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('$\mu2$ pT [GeV]')
plt.ylabel('n_ev')
plt.show()

sb.distplot(mu_wrong['mu2_eta'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_wrong['mu2_eta'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('$\mu2$ eta')
plt.ylabel('n_ev')
plt.show()

sb.distplot(mu_wrong['mu2_phi'], kde=False, color='g', norm_hist=True, label='$\mu$ channel')
sb.distplot(tau_wrong['mu2_phi'], kde=False, color='r', norm_hist=True, label='$\tau$ channel')
plt.xlabel('$\mu2$ phi')
plt.ylabel('n_ev')
plt.show()